In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression



In [ ]:
adi_god = pd.read_csv('imputed_city_hour_scaled.csv')

adi_god.head()

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Aizawl,2020-03-11 09:00:00,0.135489,0.427701,0.032890,0.198269,0.088784,0.188586,0.006954,0.069510,0.081963,0.018027,0.402398,0.057374,54.0,Satisfactory
1,Aizawl,2020-03-11 10:00:00,0.119910,0.278934,0.034848,0.112310,0.068482,0.196510,0.010380,0.050063,0.062373,0.009014,0.202416,0.057374,54.0,Satisfactory
2,Amaravati,2017-11-24 19:00:00,0.204504,0.293813,0.018839,0.241656,0.097427,0.095732,0.003023,0.087704,0.906653,0.002084,0.078226,0.004398,155.0,Moderate
3,Amaravati,2017-11-24 20:00:00,0.225354,0.324166,0.017781,0.197787,0.081307,0.105476,0.003023,0.084439,0.869060,0.002605,0.080872,0.004398,155.0,Moderate
4,Amaravati,2017-11-24 21:00:00,0.235035,0.328256,0.014421,0.177526,0.071980,0.112626,0.003023,0.074857,0.840547,0.003126,0.078738,0.003682,155.0,Moderate


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [6]:


# Load the data
data = pd.read_csv("imputed_city_hour_scaled.csv") 
# Preprocessing
data['City'] = LabelEncoder().fit_transform(data['City'])
data['Datetime'] = pd.to_datetime(data['Datetime'])  # Ensure 'Datetime' is in datetime format
data['hour'] = data['Datetime'].dt.hour
data['month'] = data['Datetime'].dt.month
data['year'] = data['Datetime'].dt.year

# Grouped Train-Test Split for Time-Series Data
def time_series_split_by_city(data, test_ratio=0.2):
    train_data = pd.DataFrame()
    test_data = pd.DataFrame()
    
    for city in data['City'].unique():
        city_data = data[data['City'] == city].sort_values(by='Datetime')  # Sort by time
        test_size = int(len(city_data) * test_ratio)
        train_data = pd.concat([train_data, city_data[:-test_size]])  # Add all except last portion
        test_data = pd.concat([test_data, city_data[-test_size:]])    # Add last portion
        
    return train_data, test_data

# Perform the split
train_data, test_data = time_series_split_by_city(data)

# Drop 'Datetime' after splitting
train_data = train_data.drop(columns=['Datetime'])
test_data = test_data.drop(columns=['Datetime'])

# Separate features and targets
X_train = train_data.drop(columns=['AQI', 'AQI_Bucket'])
y_reg_train = train_data['AQI']
y_clf_train = LabelEncoder().fit_transform(train_data['AQI_Bucket'])

X_test = test_data.drop(columns=['AQI', 'AQI_Bucket'])
y_reg_test = test_data['AQI']
y_clf_test = LabelEncoder().fit_transform(test_data['AQI_Bucket'])

# Normalize features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Regression Model
reg_model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

reg_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
reg_model.fit(X_train, y_reg_train, epochs=1, batch_size=32, validation_split=0.2)

# Predict and evaluate regression
y_reg_pred = reg_model.predict(X_test)
print("MAE:", mean_absolute_error(y_reg_test, y_reg_pred))

# Classification Model
clf_model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_clf_train)), activation='softmax')  # Output layer for classification
])

clf_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
clf_model.fit(X_train, y_clf_train, epochs=50, batch_size=32, validation_split=0.2)

# Predict and evaluate classification
y_clf_pred = np.argmax(clf_model.predict(X_test), axis=1)
print("Confusion Matrix:")
print(confusion_matrix(y_clf_test, y_clf_pred))
print("Classification Report:")
print(classification_report(y_clf_test, y_clf_pred))


C:\Users\ADITYA UPADHYAY\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5724/5724 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 7873.4688 - mae: 59.6369 - val_loss: 6036.1875 - val_mae: 59.8808
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
MAE: 26.798043458461677
Epoch 1/50


C:\Users\ADITYA UPADHYAY\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1372/5724 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.5057 - loss: 1.2154

KeyboardInterrupt: 

In [9]:
from sklearn.cluster import KMeans
import numpy as np

cpy = adi_god.copy()
#Label Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cpy['City'] = le.fit_transform(cpy['City'])
cpy['Datetime'] = le.fit_transform(cpy['Datetime'])
cpy["AQI_Bucket"] = le.fit_transform(cpy["AQI_Bucket"])
# Assuming `data` is a NumPy array
kmeans = KMeans(n_clusters=10000, random_state=42)
kmeans.fit(cpy)
reduced_data = kmeans.cluster_centers_  # Cluster centroids as reduced data


In [10]:

# save the model to disk
import pickle
filename = 'kmeans_model.sav'
pickle.dump(kmeans, open(filename, 'wb'))

#also save the centeres
filename = 'kmeans_centers.sav'
pickle.dump(reduced_data, open(filename, 'wb'))



In [24]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('imputed_city_hour_scaled.csv')
# Assuming your DataFrame is named 'df' and 'Datetime' is a datetime column
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Initialize label encoder for encoding later
le = LabelEncoder()
df['City'] = le.fit_transform(df['City'])
df['AQI_Bucket'] = le.fit_transform(df['AQI_Bucket'])
# Resample to daily average values for each city
df_resampled = (
    df.set_index('Datetime')
    .groupby('City')
    .resample('D')
    .mean()
)

# Encode 'City' and 'AQI_Bucket' columns after resampling
df_resampled["City"] = le.fit_transform(df_resampled["City"])
df_resampled["AQI_Bucket"] = le.fit_transform(df_resampled["AQI_Bucket"])

print(df_resampled.shape)


(14322, 15)


In [21]:
from scipy.signal import find_peaks
df = pd.read_csv('imputed_city_hour_scaled.csv')
# Detect peaks in PM2.5 for each city
for city in df['City'].unique():
    city_data = df[df['City'] == city]
    peaks, _ = find_peaks(city_data['PM2.5'], height=0.01)  # Example threshold

    # Keep only rows corresponding to the peaks
    df_peaks = city_data.iloc[peaks]

print(df_peaks.head())
print(df_peaks.shape)

                 City             Datetime     PM2.5      PM10        NO  \
252400  Visakhapatnam  2016-07-03 08:00:00  0.163051  0.236337  0.105099   
252476  Visakhapatnam  2016-07-06 12:00:00  0.066990  0.169604  0.090281   
252486  Visakhapatnam  2016-07-06 22:00:00  0.119116  0.250545  0.241659   
252489  Visakhapatnam  2016-07-07 01:00:00  0.134009  0.262600  0.093007   
252498  Visakhapatnam  2016-07-07 10:00:00  0.092556  0.169820  0.156749   

             NO2       NOx       NH3        CO       SO2        O3   Benzene  \
252400  0.561341  0.241930  0.263279  0.049582  0.242814  0.247320  0.138801   
252476  0.293213  0.146271  0.159891  0.029124  0.136797  0.263124  0.059709   
252486  0.555793  0.317367  0.119808  0.042225  0.063264  0.096815  0.141718   
252489  0.397895  0.180965  0.107343  0.028419  0.047886  0.152702  0.085500   
252498  0.404378  0.220241  0.094910  0.022070  0.021979  0.190745  0.078466   

         Toluene    Xylene   AQI    AQI_Bucket  
252400  0.183